# Recap

### Main

In [1]:
%%writefile main.kv

<Widget>:
    canvas:
        Color:
            rgba: .5,.5,.5,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2

Overwriting main.kv


In [2]:
%%writefile main.py

from kivy.app import App
from selectiontool import SelectionTool


class MainApp(App):
    title = 'Image Selection Tool'

    def build(self):
        return SelectionTool()


if __name__ == '__main__':
    MainApp().run()

Overwriting main.py


### ImagePane

In [3]:
%%writefile imagepane.kv

<ImagePane>:
    allow_stretch: True

Overwriting imagepane.kv


In [4]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App

from selectionbox import SelectionBox

Builder.load_file("imagepane.kv")


class ImagePane(Image):

    drawing_rectangle = None
    rectangles = []

    def on_touch_move(self, touch):
        if self.collide_point(*touch.pos):
            pos = [min(touch.pos[n], touch.opos[n]) for n in [0, 1]]
            size = [abs(touch.pos[n] - touch.opos[n]) for n in [0, 1]]
            if self.drawing_rectangle is None:
                self.drawing_rectangle = SelectionBox(pos=pos, size=size, image_pane=self)
                self.add_new_rectangle(self.drawing_rectangle)
            else:
                self.drawing_rectangle.pos = pos
                self.drawing_rectangle.size = size

    def on_touch_up(self, touch):
        if self.drawing_rectangle:
            self.drawing_rectangle.compute_unit_coordinates()
            self.drawing_rectangle = None
            self.store_rectangles()

    def add_new_rectangle(self, rect):
        self.add_widget(rect)
        self.rectangles.append(rect)
        
    def delete_last_rectangle(self):
        if self.rectangles:
            bad_rectangle = self.rectangles.pop()
            self.remove_widget(bad_rectangle)
            self.store_rectangles()
            
    def clear_rectangles(self):
        self.rectangles = []
        self.clear_widgets()

    def store_rectangles(self):
        App.get_running_app().root.store_rectangles(self.rectangles)


Overwriting imagepane.py


### SelectionBox

In [5]:
%%writefile selectionbox.kv

<SelectionBox>:
    label: _label
    color: (1, 0, 0, 1)
    image_pane: None
    canvas:
        Color:
            rgba: root.color
        Line:
            width: 2.
            rectangle: (self.x, self.y, self.width, self.height)
    TextInput:
        id: _label
        multiline: False
        size_hint: 1, None
        height: 50
        center: root.center
        on_text: root.image_pane.store_rectangles() if root.image_pane else None


Overwriting selectionbox.kv


In [6]:
%%writefile selectionbox.py

from kivy.uix.widget import Widget
from kivy.lang import Builder

Builder.load_file('selectionbox.kv')


class SelectionBox(Widget):

    def __init__(self, image_pane, text='', pos=None, size=None, unit_pos=None, unit_size=None):
        super(SelectionBox, self).__init__(pos=pos, size=size)
        self.label.text = text
        self.image_pane = image_pane
        self.unit_pos = unit_pos
        self.unit_size = unit_size
        
    def to_dict(self):
        return {'text': self.label.text, 'pos': self.pos, 'size': self.size,
                'unit_pos': self.unit_pos, 'unit_size': self.unit_size}

    def compute_unit_coordinates(self):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.unit_pos = tuple([(self.pos[n] - image_pos[n]) * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])
        self.unit_size = tuple([self.size[n] * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])

    def compute_screen_coordinates(self, *_):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.pos = [self.unit_pos[n] * self.image_pane.norm_image_size[n] + image_pos[n] for n in [0, 1]]
        self.size = [self.unit_size[n] * self.image_pane.norm_image_size[n] for n in [0, 1]]


Overwriting selectionbox.py


### SelectionTool

In [7]:
%%writefile selectiontool.kv

<SelectionTool>:
    orientation: "vertical"

    book_selector: _book_selector
    page_selector: _page_selector
    image_pane: _image_pane
    word_list: _word_list

    book_id: self.book_selector.text
    page: self.page_selector.text

    BoxLayout:
        orientation: 'horizontal'

        Widget: # Spacer
            size_hint_x: 4

        Spinner:
            id: _book_selector

        Spinner:
            id: _page_selector

    Button:
        text: "Delete Last Rectangle"
        on_press: root.image_pane.delete_last_rectangle()

    BoxLayout:
        orientation: 'horizontal'
        size_hint_y: 16
        BoxLayout:
            id: _word_list
            size_hint_x: 0.2
            orientation: 'vertical'

        ImagePane:
            id: _image_pane
            size_hint_x: 0.8
            source: ''

Overwriting selectiontool.kv


In [8]:
%%writefile selectiontool.py

import os
import glob
import json

from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.properties import StringProperty
from kivy.lang import Builder

from imagepane import ImagePane
from selectionbox import SelectionBox

Builder.load_file('selectiontool.kv')


class SelectionTool(BoxLayout):
    library_directory = 'Example_Data'

    book_id = StringProperty()
    page = StringProperty()

    def __init__(self):
        super(SelectionTool, self).__init__()
        self.all_rectangles = {}
        self.rectangles_filename = 'rectangles.json'
        self.load_rectangles()
        
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

        self.on_book_id()

    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Images'

        self.word_list.clear_widgets()
        with open(os.path.join(self.library_directory, self.book_id, 'word_list.txt')) as fp:
            for word in sorted(fp.readlines()):
                self.word_list.add_widget(Label(text=word.strip()))
        
        self.on_page()
    
    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        self.image_pane.clear_rectangles()
        try:
            for rect in self.all_rectangles[self.book_id][self.page]:
                self.image_pane.add_new_rectangle(rect)
        except KeyError:
            pass

    def store_rectangles(self, rectangles):
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rectangles]})
        self.save_rectangles()

    def load_rectangles(self):
        self.all_rectangles = {}
        try:
            with open(self.rectangles_filename) as fd:
                all_rectangles_dict = json.load(fd)
                for book_id, book_rectangles in all_rectangles_dict.items():
                    self.all_rectangles[book_id] = {}
                    for page, rectangles in book_rectangles.items():
                        self.all_rectangles[book_id][page] = \
                            [SelectionBox(image_pane=self.image_pane, **rect) for rect in rectangles]
        except IOError:
            print("Can't find rectangles file!")
        
    def save_rectangles(self):
        rectangle_dict = {}
        for book_id, book_rectangles in self.all_rectangles.items():
            for page, rectangles in book_rectangles.items():
                page_dict = {page: [rect.to_dict() for rect in rectangles]}
                if rectangles:
                    rectangle_dict[book_id] = rectangle_dict.get(book_id, {})
                    rectangle_dict[book_id].update(page_dict)
        
        with open(self.rectangles_filename, 'w') as fd:
            json.dump(rectangle_dict, fd, sort_keys=True, indent=4, separators=(',', ': '))


Overwriting selectiontool.py


In [9]:
#!python main.py

# Resizing rectangles

Note that if we resize the window, the rectangles aren't where they should be. What happens when we resize? The overall `BoxLayout` is resized, so Kivy resizes all the `Widgets` inside it, including other `BoxLayouts` that resize their children. 

The `ImagePane` is an `Image`, so it knows to resize the image. But the `ImagePane` doesn't know it should do anything with the `SelectionBoxes`. We have to tell it what to do.

When do we have to redraw the rectangles? If the `ImagePane` changes its size or position on the screen.

In [10]:
%%writefile imagepane.kv

<ImagePane>:
    allow_stretch: True
    on_size: self.redraw_rectangles()
    on_pos: self.redraw_rectangles()


Overwriting imagepane.kv


In [11]:
%%writefile -a imagepane.py

    def redraw_rectangles(self):
        for rect in self.rectangles:
            rect.compute_screen_coordinates()


Appending to imagepane.py


Note that `compute_screen_coordinates()` just computes the screen size and position of the rectangle:
```
def compute_screen_coordinates(self, *_):
    image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                 for n in (0, 1)]
    self.pos = [self.unit_pos[n] * self.image_pane.norm_image_size[n] + image_pos[n] for n in [0, 1]]
    self.size = [self.unit_size[n] * self.image_pane.norm_image_size[n] for n in [0, 1]]
```
But `size` and `pos` are Kivy properties, and in the kv file `selectionbox.kv` we have
```
<SelectionBox>:
    label: _label
    color: (1, 0, 0, 1)
    image_pane: None
    canvas:
        Color:
            rgba: root.color
        Line:
            width: 2.
            rectangle: (self.x, self.y, self.width, self.height)
    ...
```
So if `pos` and `size` change, `x`, `y`, `width` and `height` will automatically change, and since they are all Kivy properties too, Kivy will change the rectangle drawn on the canvas!

If we resize the `ImagePane` and then change pages, we have to make sure that we draw the new rectangles appropriately.

In [12]:
%%writefile -a selectiontool.py
            
    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        self.image_pane.clear_rectangles()
        try:
            for rect in self.all_rectangles[self.book_id][self.page]:
                rect.compute_screen_coordinates()  # added
                self.image_pane.add_new_rectangle(rect)
        except KeyError:
            pass
        

Appending to selectiontool.py


In [13]:
#!python main.py

<img src="Images/resized_rectangles.png"/>

# Show which words we've made boxes for

Finally, let's indicate which words in the list we've made boxes for.

In [14]:
%%writefile -a selectiontool.py

    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Images'

        self.word_list.clear_widgets()
        with open(os.path.join(self.library_directory, self.book_id, 'word_list.txt')) as fp:
            for word in sorted(fp.readlines()):
                self.word_list.add_widget(Label(text=word.strip()))
        self.color_word_list()  # added
        self.on_page()
        
    def store_rectangles(self, rectangles):
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rectangles]})
        self.color_word_list()  # added
        self.save_rectangles()
        
    def color_word_list(self):
        if self.book_id in self.all_rectangles:
            rectangle_labels = [rect.label.text for page_rects in self.all_rectangles[self.book_id].values()
                                for rect in page_rects]
        else:
            rectangle_labels = []
            
        for label in self.word_list.children:
            label.color = (0, 1, 0, 1) if label.text in rectangle_labels else (1, 1, 1, 1)


Appending to selectiontool.py


In [15]:
#!python main.py

<img src="Images/coloured_word_list.png"/>

# Make our own events

Recall that in `imagepane.py` we had:
```python
    def store_rectangles(self):
        App.get_running_app().root.store_rectangles(self.rectangles)
```

This is a bit clunky, and assumes that the root widget is a `SelectionTool`. But what if we build a tool that had a `SelectionTool` as well as another tool, both in a `BoxLayout`? Then `store_rectangles()` would crash, complaining that `BoxLayout` didn't contain a `store_rectangles()` method.

A better way to call the `store_rectangles()` method in `SelectionTool` is to have `ImagePane` dispatch an event, and have `SelectionTool` react to the event.

In [16]:
%%writefile -a imagepane.py

    def __init__(self, **kwargs):
        super(ImagePane, self).__init__(**kwargs)
        self.register_event_type('on_store_rectangles')  # tell Kivy that we've going to dispatch this event

    def store_rectangles(self):
        self.dispatch('on_store_rectangles', rectangles=self.rectangles)  # send an event with an argument

    def on_store_rectangles(self, *args, **kwargs):  # template for the event. We shouldn't have to do this. :( 
        pass
    

Appending to imagepane.py


In [17]:
%%writefile -a selectiontool.py

    def __init__(self):
        super(SelectionTool, self).__init__()
        
        self.image_pane.bind(on_store_rectangles=self.store_rectangles)  # map the event to a function
        
        self.all_rectangles = {}
        self.rectangles_filename = 'rectangles.json'
        self.load_rectangles()
        
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

        self.on_book_id()

    def store_rectangles(self, sender=None, rectangles=[]):  # 'sender': events say what object dispatched them
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rectangles]})
        self.color_word_list()
        self.save_rectangles()
  

Appending to selectiontool.py


In [18]:
#!python main.py